# Day 3

Associated learning goal: Preprocess image data without introducing bias in downstream analysis.

- Fix shot noise in the image
    - using a median filter
- Background subtraction
    - manual (as a strawman)
    - Ostu
    - rolling ball (show a failure)
- Discuss the concept of a pre-processing pipeline
- Order the pre-processing steps for the pipeline based on the goals of the project
- Masks and morphological operations
    - Show slicing an image with a mask
    - Erode/dilate
    - Open/close

First some boilerplate code to make it easier to access useful libraries, and to make it easier to visualize data in the notebook.

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Image processing pipelines: an example

TODO: before and after of an image with shot noise, background objects, and uneven illumination.

We want to view the fluorescence intensity of cells in this image. We will need to reliably find cell bodies and be careful not to adjust their pixel intensities.

**Preprocessing misteps are a good way to get a paper retracted. We argue that it's easier to make these misteps when doing things manually, but it's not *impossible* to do it computationally. In fact if you don't check intermediate steps of your data in either case, it's no good. Always visualize your intermediates.**